In [1]:
# Import libraries
import pandas as pd
import datetime
import re
import urllib.request
import urllib.parse
import xml.dom.minidom

In [2]:
# Method for extracting text from xml file
def getText(nodelist):
    rc = []
    for node in nodelist:
        if node.nodeType == node.TEXT_NODE:
            rc.append(node.data)
    return ''.join(rc)

In [3]:
# Import Airports dataset
airports = pd.read_csv('airports_cleaned.csv')

In [4]:
# Add Declination column to dataset
airports['DECLINATION'] = ''

In [5]:
# Set date as now for API call
now = datetime.datetime.now()
day = now.day
month = now.month
year = now.year
# This will need to be adjusted for incidents to include date of individual incidents and merged into for loop

In [6]:
# Set base URL for query
url = "https://www.ngdc.noaa.gov/geomag-web/calculators/calculateDeclination?browserRequest=true&magneticComponent=d&"

In [7]:
# Iterate through the dataframe to calculate declination for each row
for i in airports.index:
    # Set hemisphere for lat/long
    if airports['LATITUDE'][i] > 0:
        latLabel = 'N'
    else:
        latLabel = 'S'
    if airports['LONGITUDE'][i] > 0:
        lonLabel = 'E'
    else:
        lonLabel = 'W'
    # Encode URL parameters
    data = urllib.parse.urlencode({'lat1': abs(airports['LATITUDE'][i]), 'lat1Hemisphere': latLabel, 'lon1': abs(airports['LONGITUDE'][i]), 'lon1Hemisphere': lonLabel, 'model': 'WMM', 'startYear': year, 'startMonth': month, 'startDay': day, 'resultFormat': 'xml'})
    url_req = url + data
    # Pass request to site
    req = urllib.request.urlopen(url_req)
    # Process XML file into object tree and get only declination info
    dom = xml.dom.minidom.parseString(req.read().decode("UTF-8"))
    myString = getText(dom.getElementsByTagName("declination")[0].childNodes)
    # Removes formatting to leave only declination value
    declination = str(re.findall(r"[-+]?\d*\.\d+|\d+", myString)[0])
    # Sets value for row
    airports.at[i, 'DECLINATION'] = declination

In [8]:
airports

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,DECLINATION
0,6TS0,TRUE,34.089220,-101.740461,NaN,5.74941
1,WY47,TRUE,42.853576,-106.336415,NaN,8.90491
2,WA08,Zwainz Farms,47.713439,-117.919989,NaN,14.44256
3,9IN7,Zupancic Fld,39.354722,-86.306944,NaN,-4.69381
4,1TE4,Zuehl,29.495509,-98.157787,NaN,3.68769
...,...,...,...,...,...,...
19889,55XA,1101 East Blvd Deer Park,29.702772,-95.101639,NaN,1.95632
19890,WA54,1001 Fourth Avenue Plaza,47.606765,-122.333737,NaN,15.44604
19891,FA10,1000 Museum,25.784228,-80.190017,NaN,-6.97982
19892,TN41,100 Aker Wood,35.772778,-84.765278,NaN,-5.33458


In [9]:
# Export dataframe to CSV
airports.to_csv('airports_cleaned_w-declination.csv')